# Advent of Code 2025

In [39]:
import java.io.File
abstract class Problem<T>(private val filename: String) {
    abstract val testResult: T
    protected var isTesting: Boolean = true
        private set
    fun run() {
        val testOutput = File("testData/$filename").useLines { solve(it) }
        if (testOutput != testResult) {
            throw IllegalStateException("Test output doesn't match expected result \n Actual: $testOutput \n Expected: $testResult")
        }
        println("Test results: Expected=$testResult Actual=$testOutput")

        isTesting = false
        val actualOutput = File("data/$filename").useLines { solve(it) }
        println("Solution: $actualOutput")
    }
    abstract fun solve(lines: Sequence<String>): T
}

## Day 1: Secret Entrance
### Part 1
You arrive at the secret entrance to the North Pole base ready to start decorating. Unfortunately, the password seems to have been changed, so you can't get in. A document taped to the wall helpfully explains:

"Due to new security protocols, the password is locked in the safe below. Please see the attached document for the new combination."

The safe has a dial with only an arrow on it; around the dial are the numbers 0 through 99 in order. As you turn the dial, it makes a small click noise as it reaches each number.

The attached document (your puzzle input) contains a sequence of rotations, one per line, which tell you how to open the safe. A rotation starts with an L or R which indicates whether the rotation should be to the left (toward lower numbers) or to the right (toward higher numbers). Then, the rotation has a distance value which indicates how many clicks the dial should be rotated in that direction.

So, if the dial were pointing at `11`, a rotation of `R8` would cause the dial to point at `19`. After that, a rotation of `L19` would cause it to point at `0`.

Because the dial is a circle, turning the dial left from `0` one click makes it point at `99`. Similarly, turning the dial right from `99` one click makes it point at `0`.

So, if the dial were pointing at `5`, a rotation of L10 would cause it to point at `95`. After that, a rotation of `R5` could cause it to point at `0`.

The dial starts by pointing at `50`.

You could follow the instructions, but your recent required official North Pole secret entrance security training seminar taught you that the safe is actually a decoy. The actual password is the number of times the dial is left pointing at `0` after any rotation in the sequence.

For example, suppose the attached document contained the following rotations:
```
L68
L30
R48
L5
R60
L55
L1
L99
R14
L82
```
Following these rotations would cause the dial to move as follows:

The dial starts by pointing at `50`.
The dial is rotated `L68` to point at `82`.
The dial is rotated `L30` to point at `52`.
The dial is rotated `R48` to point at `0`.
The dial is rotated `L5` to point at `95`.
The dial is rotated `R60` to point at `55`.
The dial is rotated `L55` to point at `0`.
The dial is rotated `L1` to point at `99`.
The dial is rotated `L99` to point at `0`.
The dial is rotated `R14` to point at `14`.
The dial is rotated `L82` to point at `32`.
Because the dial points at `0` a total of three times during this process, the password in this example is `3`.

Analyze the rotations in your attached document. What's the actual password to open the door?

In [2]:
class DayOnePartOne(override val testResult: Int = 3) : Problem<Int>("day1.txt") {
    override fun solve(lines: Sequence<String>): Int = lines.runningFold(50) { acc, line ->
        when (line.first()) {
            'R' -> acc + line.drop(1).toInt()
            'L' -> acc - line.drop(1).toInt()
            else -> acc
        }
    }.count { it % 100 == 0 }
}

DayOnePartOne().run()

Test results: Expected=3 Actual=3
Solution: 1139


### Part 2
You're sure that's the right password, but the door won't open. You knock, but nobody answers. You build a snowman while you think.

As you're rolling the snowballs for your snowman, you find another security document that must have fallen into the snow:

"Due to newer security protocols, please use password method 0x434C49434B until further notice."

You remember from the training seminar that "method 0x434C49434B" means you're actually supposed to count the number of times any click causes the dial to point at 0, regardless of whether it happens during a rotation or at the end of one.

Following the same rotations as in the above example, the dial points at zero a few extra times during its rotations:

The dial starts by pointing at `50`.
The dial is rotated `L68` to point at `82`; during this rotation, it points at `0` once.
The dial is rotated `L30` to point at `52`.
The dial is rotated `R48` to point at `0`.
The dial is rotated `L5` to point at `95`.
The dial is rotated `R60` to point at `55`; during this rotation, it points at `0` once.
The dial is rotated `L55` to point at `0`.
The dial is rotated `L1` to point at `99`.
The dial is rotated `L99` to point at `0`.
The dial is rotated `R14` to point at `14`.
The dial is rotated `L82` to point at `32`; during this rotation, it points at `0` once.
In this example, the dial points at `0` three times at the end of a rotation, plus three more times during a rotation. So, in this example, the new password would be `6`.

Be careful: if the dial were pointing at `50`, a single rotation like `R1000` would cause the dial to point at `0` ten times before returning back to `50`!

Using password method 0x434C49434B, what is the password to open the door?

In [3]:
class DayOnePartTwo(override val testResult: Int = 6) : Problem<Int>("day1.txt") {
    override fun solve(lines: Sequence<String>): Int {
        var current = 50
        var zeroCount = 0
        lines.forEach { line ->
            when (line.first()) {
                'R' -> {
                    repeat(line.drop(1).toInt()) {
                        current++
                        if (current == 100) {
                            zeroCount++
                            current = 0
                        }
                    }
                }
                'L' -> {
                    repeat(line.drop(1).toInt()) {
                        current--
                        when (current) {
                            0 -> zeroCount++
                            -1 -> current = 99
                        }
                    }
                }
            }
        }
        return zeroCount
    }
}

DayOnePartTwo().run()

Test results: Expected=6 Actual=6
Solution: 6684


## Day 2: Invalid IDs
### Part 1
You get inside and take the elevator to its only other stop: the gift shop. "Thank you for visiting the North Pole!" gleefully exclaims a nearby sign. You aren't sure who is even allowed to visit the North Pole, but you know you can access the lobby through here, and from there you can access the rest of the North Pole base.

As you make your way through the surprisingly extensive selection, one of the clerks recognizes you and asks for your help.

As it turns out, one of the younger Elves was playing on a gift shop computer and managed to add a whole bunch of invalid product IDs to their gift shop database! Surely, it would be no trouble for you to identify the invalid product IDs for them, right?

They've even checked most of the product ID ranges already; they only have a few product ID ranges (your puzzle input) that you'll need to check. For example:

```
11-22,95-115,998-1012,1188511880-1188511890,222220-222224,
1698522-1698528,446443-446449,38593856-38593862,565653-565659,
824824821-824824827,2121212118-2121212124
```
(The ID ranges are wrapped here for legibility; in your input, they appear on a single long line.)

The ranges are separated by commas (,); each range gives its first ID and last ID separated by a dash (-).

Since the young Elf was just doing silly patterns, you can find the invalid IDs by looking for any ID which is made only of some sequence of digits repeated twice. So, `55` (5 twice), `6464` (64 twice), and `123123` (123 twice) would all be invalid IDs.

None of the numbers have leading zeroes; `0101` isn't an ID at all. (`101` is a valid ID that you would ignore.)

Your job is to find all of the invalid IDs that appear in the given ranges. In the above example:

`11-22` has two invalid IDs, `11` and `22`.
`95-115` has one invalid ID, `99`.
`998-1012` has one invalid ID, `1010`.
`1188511880-1188511890` has one invalid ID, `1188511885`.
`222220-222224` has one invalid ID, `222222`.
`1698522-1698528` contains no invalid IDs.
`446443-446449` has one invalid ID, `446446`.
`38593856-38593862` has one invalid ID, `38593859`.
The rest of the ranges contain no invalid IDs.
Adding up all the invalid IDs in this example produces `1227775554`.

What do you get if you add up all of the invalid IDs?

In [4]:
class DayTwoPartOne(override val testResult: Long = 1227775554L) : Problem<Long>("day2.txt") {
    override fun solve(lines: Sequence<String>): Long {
        var acc = 0L
        lines
            .first()
            .split(",")
            .map { it.substringBefore("-") to it.substringAfter("-") }
            .forEach { split ->
                for (i in split.first.toLong()..split.second.toLong()) {
                    val str = i.toString()
                    if (str.length % 2 == 0 && str.take(str.length / 2) == str.takeLast(str.length / 2)) {
                        acc += i
                    }
                }
            }
        return acc
    }
}

DayTwoPartOne().run()

Test results: Expected=1227775554 Actual=1227775554
Solution: 23534117921


### Part 2
The clerk quickly discovers that there are still invalid IDs in the ranges in your list. Maybe the young Elf was doing other silly patterns as well?

Now, an ID is invalid if it is made only of some sequence of digits repeated at least twice. So, `12341234` (`1234` two times), `123123123` (`123` three times), `1212121212` (`12` five times), and `1111111` (`1` seven times) are all invalid IDs.

From the same example as before:

`11-22` still has two invalid IDs, `11` and `22`.
`95-115` now has two invalid IDs, `99` and `111`.
`998-1012` now has two invalid IDs, `999` and `1010`.
`1188511880-1188511890` still has one invalid ID, `1188511885`.
`222220-222224` still has one invalid ID, `222222`.
`1698522-1698528` still contains no invalid IDs.
`446443-446449` still has one invalid ID, `446446`.
`38593856-38593862` still has one invalid ID, `38593859`.
`565653-565659` now has one invalid ID, `565656`.
`824824821-824824827` now has one invalid ID, `824824824`.
`2121212118-2121212124` now has one invalid ID, `121212121`.
Adding up all the invalid IDs in this example produces `4174379265`.

What do you get if you add up all of the invalid IDs using these new rules?

In [5]:
class DayTwoPartTwo(override val testResult: Long = 4174379265) : Problem<Long>("day2.txt") {
    override fun solve(lines: Sequence<String>): Long {
        var acc = 0L
        lines
            .first()
            .split(",")
            .map { it.substringBefore("-") to it.substringAfter("-") }
            .forEach { split ->
                for (i in split.first.toLong()..split.second.toLong()) {
                    if (i < 10) continue
                    val str = i.toString()
                    if (str.all { it == str.first() }) {
                        acc += i
                        continue
                    }

                    for (j in (str.length / 2) downTo 2) {
                        val chunked = str.chunked(j)
                        if (chunked.size > 1 && chunked.all { it == chunked.first() }) {
                            acc += i
                            break
                        }
                    }
                }
        }
        return acc
    }
}

DayTwoPartTwo().run()

Test results: Expected=4174379265 Actual=4174379265
Solution: 31755323497


## Day 3: Escalator batteries

### Part 1
You descend a short staircase, enter the surprisingly vast lobby, and are quickly cleared by the security checkpoint. When you get to the main elevators, however, you discover that each one has a red light above it: they're all offline.

"Sorry about that," an Elf apologizes as she tinkers with a nearby control panel. "Some kind of electrical surge seems to have fried them. I'll try to get them online soon."

You explain your need to get further underground. "Well, you could at least take the escalator down to the printing department, not that you'd get much further than that without the elevators working. That is, you could if the escalator weren't also offline."

"But, don't worry! It's not fried; it just needs power. Maybe you can get it running while I keep working on the elevators."

There are batteries nearby that can supply emergency power to the escalator for just such an occasion. The batteries are each labeled with their joltage rating, a value from `1` to `9`. You make a note of their joltage ratings (your puzzle input). For example:
```
987654321111111
811111111111119
234234234234278
818181911112111
```
The batteries are arranged into banks; each line of digits in your input corresponds to a single bank of batteries. Within each bank, you need to turn on exactly two batteries; the joltage that the bank produces is equal to the number formed by the digits on the batteries you've turned on. For example, if you have a bank like `12345` and you turn on batteries `2` and `4`, the bank would produce `24` jolts. (You cannot rearrange batteries.)

You'll need to find the largest possible joltage each bank can produce. In the above example:

In `987654321111111`, you can make the largest joltage possible, `98`, by turning on the first two batteries.
In `811111111111119`, you can make the largest joltage possible by turning on the batteries labeled `8` and `9`, producing `89` jolts.
In `234234234234278`, you can make `78` by turning on the last two batteries (marked `7` and `8`).
In `818181911112111`, the largest joltage you can produce is `92`.
The total output joltage is the sum of the maximum joltage from each bank, so in this example, the total output joltage is `98` + `89` + `78` + `92` = `357`.

There are many batteries in front of you. Find the maximum joltage possible from each bank; what is the total output joltage?

In [6]:
class DayThreePartOne(override val testResult: Int = 357) : Problem<Int>("day3.txt") {
    override fun solve(lines: Sequence<String>): Int = lines.sumOf { line ->
        val batteries = line.map { it.digitToInt() }
        val first = batteries.dropLast(1).withIndex().maxBy { it.value }
        val second = batteries.drop(first.index + 1).max()
        "${first.value}$second".toInt()
    }
}

DayThreePartOne().run()

Test results: Expected=357 Actual=357
Solution: 16973


### Part 2
The escalator doesn't move. The Elf explains that it probably needs more joltage to overcome the static friction of the system and hits the big red "joltage limit safety override" button. You lose count of the number of times she needs to confirm "yes, I'm sure" and decorate the lobby a bit while you wait.

Now, you need to make the largest joltage by turning on exactly twelve batteries within each bank.

The joltage output for the bank is still the number formed by the digits of the batteries you've turned on; the only difference is that now there will be 12 digits in each bank's joltage output instead of two.

Consider again the example from before:

987654321111111
811111111111119
234234234234278
818181911112111
Now, the joltages are much larger:

In 987654321111111, the largest joltage can be found by turning on everything except some 1s at the end to produce 987654321111.
In the digit sequence 811111111111119, the largest joltage can be found by turning on everything except some 1s, producing 811111111119.
In 234234234234278, the largest joltage can be found by turning on everything except a 2 battery, a 3 battery, and another 2 battery near the start to produce 434234234278.
In 818181911112111, the joltage 888911112111 is produced by turning on everything except some 1s near the front.
The total output joltage is now much larger: 987654321111 + 811111111119 + 434234234278 + 888911112111 = 3121910778619.

What is the new total output joltage?

In [7]:
class DayThreePartTwo(override val testResult: Long = 3121910778619L) : Problem<Long>("day3.txt") {
    override fun solve(lines: Sequence<String>): Long = lines.sumOf { line ->
        val batteries = line.map { it.digitToInt() }
        var acc = ""
        var indexOfLastBatteryUsed = -1
        for (i in 11 downTo 0) {
            val battery = batteries.withIndex().drop(indexOfLastBatteryUsed + 1).dropLast(i).maxBy { it.value }
            indexOfLastBatteryUsed = battery.index
            acc += battery.value
        }
        acc.toLong()
    }
}

DayThreePartTwo().run()

Test results: Expected=3121910778619 Actual=3121910778619
Solution: 168027167146027


## Day 4: Paper rolls
### Part 1
You ride the escalator down to the printing department. They're clearly getting ready for Christmas; they have lots of large rolls of paper everywhere, and there's even a massive printer in the corner (to handle the really big print jobs).

Decorating here will be easy: they can make their own decorations. What you really need is a way to get further into the North Pole base while the elevators are offline.

"Actually, maybe we can help with that," one of the Elves replies when you ask for help. "We're pretty sure there's a cafeteria on the other side of the back wall. If we could break through the wall, you'd be able to keep moving. It's too bad all of our forklifts are so busy moving those big rolls of paper around."

If you can optimize the work the forklifts are doing, maybe they would have time to spare to break through the wall.

The rolls of paper (`@`) are arranged on a large grid; the Elves even have a helpful diagram (your puzzle input) indicating where everything is located.

For example:
```
..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@.
```
The forklifts can only access a roll of paper if there are fewer than four rolls of paper in the eight adjacent positions. If you can figure out which rolls of paper the forklifts can access, they'll spend less time looking and more time breaking down the wall to the cafeteria.

In this example, there are `13` rolls of paper that can be accessed by a forklift (marked with x):
```
..xx.xx@x.
x@@.@.@.@@
@@@@@.x.@@
@.@@@@..@.
x@.@@@@.@x
.@@@@@@@.@
.@.@.@.@@@
x.@@@.@@@@
.@@@@@@@@.
x.x.@@@.x.
```
Consider your complete diagram of the paper roll locations. How many rolls of paper can be accessed by a forklift?

In [8]:
class DayFourPartOne(override val testResult: Int = 13) : Problem<Int>("day4.txt") {
    private val d = listOf(
        -1 to -1,
        -1 to 0,
        -1 to 1,
        0 to -1,
        0 to 1,
        1 to -1,
        1 to 0,
        1 to 1,
    )

    override fun solve(lines: Sequence<String>): Int {
        val lineList = lines.toList()
        val height = lineList.size + 2
        val width = lineList[0].length + 2
        val matrix = Array(height) { row ->
            Array(width) { column ->
                if (row == 0 || row == height - 1 || column == 0 || column == width - 1) {
                    EMPTY
                } else {
                    lineList[row - 1][column - 1]
                }
            }
        }

        var acc = 0
        for (row in 1..(matrix.size - 2)) {
            for (col in 1..(matrix[0].size - 2)) {
                if (matrix[row][col] == ROLL && d.sumOf { (i, j) -> if (matrix[row + i][col + j] == ROLL) 1 else 0 } < 4) {
                    acc++
                }
            }
        }
        return acc
    }

    companion object {
        private const val ROLL = '@'
        private const val EMPTY = '.'
    }
}

DayFourPartOne().run()

Test results: Expected=13 Actual=13
Solution: 1397


class DayFourPartTwo(override val testResult: Int = 43) : Problem<Int>("day4.txt") {
    private val d = listOf(
        -1 to -1,
        -1 to 0,
        -1 to 1,
        0 to -1,
        0 to 1,
        1 to -1,
        1 to 0,
        1 to 1,
    )

    override fun solve(lines: Sequence<String>): Int {
        val lineList = lines.toList()
        val height = lineList.size + 2
        val width = lineList[0].length + 2
        val matrix = Array(height) { row ->
            Array(width) { column ->
                if (row == 0 || row == height - 1 || column == 0 || column == width - 1) {
                    EMPTY
                } else {
                    lineList[row - 1][column - 1]
                }
            }
        }

        var acc = 0
        var updates = 0
        do {
            updates = 0
            for (row in 1..(matrix.size - 2)) {
                for (col in 1..(matrix[0].size - 2)) {
                    if (matrix[row][col] == ROLL && d.sumOf { (i, j) -> if (matrix[row + i][col + j] == ROLL) 1 else 0 } < 4) {
                        matrix[row][col] = EMPTY
                        updates++
                        acc++
                    }
                }
            }
        } while (updates != 0)
        return acc
    }

    companion object {
        private const val ROLL = '@'
        private const val EMPTY = '.'
    }
}

DayFourPartTwo().run()

### Part 2
Now, the Elves just need help accessing as much of the paper as they can.

Once a roll of paper can be accessed by a forklift, it can be removed. Once a roll of paper is removed, the forklifts might be able to access more rolls of paper, which they might also be able to remove. How many total rolls of paper could the Elves remove if they keep repeating this process?

Starting with the same example as above, here is one way you could remove as many rolls of paper as possible, using highlighted @ to indicate that a roll of paper is about to be removed, and using x to indicate that a roll of paper was just removed:
```
Initial state:
..@@.@@@@.
@@@.@.@.@@
@@@@@.@.@@
@.@@@@..@.
@@.@@@@.@@
.@@@@@@@.@
.@.@.@.@@@
@.@@@.@@@@
.@@@@@@@@.
@.@.@@@.@.

Remove 13 rolls of paper:
..xx.xx@x.
x@@.@.@.@@
@@@@@.x.@@
@.@@@@..@.
x@.@@@@.@x
.@@@@@@@.@
.@.@.@.@@@
x.@@@.@@@@
.@@@@@@@@.
x.x.@@@.x.

Remove 12 rolls of paper:
.......x..
.@@.x.x.@x
x@@@@...@@
x.@@@@..x.
.@.@@@@.x.
.x@@@@@@.x
.x.@.@.@@@
..@@@.@@@@
.x@@@@@@@.
....@@@...

Remove 7 rolls of paper:
..........
.x@.....x.
.@@@@...xx
..@@@@....
.x.@@@@...
..@@@@@@..
...@.@.@@x
..@@@.@@@@
..x@@@@@@.
....@@@...

Remove 5 rolls of paper:
..........
..x.......
.x@@@.....
..@@@@....
...@@@@...
..x@@@@@..
...@.@.@@.
..x@@.@@@x
...@@@@@@.
....@@@...

Remove 2 rolls of paper:
..........
..........
..x@@.....
..@@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@x.
....@@@...

Remove 1 roll of paper:
..........
..........
...@@.....
..x@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...

Remove 1 roll of paper:
..........
..........
...x@.....
...@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...

Remove 1 roll of paper:
..........
..........
....x.....
...@@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...

Remove 1 roll of paper:
..........
..........
..........
...x@@....
...@@@@...
...@@@@@..
...@.@.@@.
...@@.@@@.
...@@@@@..
....@@@...
```
Stop once no more rolls of paper are accessible by a forklift. In this example, a total of 43 rolls of paper can be removed.

Start with your original diagram. How many rolls of paper in total can be removed by the Elves and their forklifts?

In [9]:
class DayFourPartTwo(override val testResult: Int = 43) : Problem<Int>("day4.txt") {
    private val d = listOf(
        -1 to -1,
        -1 to 0,
        -1 to 1,
        0 to -1,
        0 to 1,
        1 to -1,
        1 to 0,
        1 to 1,
    )

    override fun solve(lines: Sequence<String>): Int {
        val lineList = lines.toList()
        val height = lineList.size + 2
        val width = lineList[0].length + 2
        val matrix = Array(height) { row ->
            Array(width) { column ->
                if (row == 0 || row == height - 1 || column == 0 || column == width - 1) {
                    EMPTY
                } else {
                    lineList[row - 1][column - 1]
                }
            }
        }

        var acc = 0
        var updates = 0
        do {
            updates = 0
            for (row in 1..(matrix.size - 2)) {
                for (col in 1..(matrix[0].size - 2)) {
                    if (matrix[row][col] == ROLL && d.sumOf { (i, j) -> if (matrix[row + i][col + j] == ROLL) 1 else 0 } < 4) {
                        matrix[row][col] = EMPTY
                        updates++
                        acc++
                    }
                }
            }
        } while (updates != 0)
        return acc
    }

    companion object {
        private const val ROLL = '@'
        private const val EMPTY = '.'
    }
}

DayFourPartTwo().run()

Test results: Expected=43 Actual=43
Solution: 8758


## Day 5: Fresh Ingredients
### Part 1
As the forklifts break through the wall, the Elves are delighted to discover that there was a cafeteria on the other side after all.

You can hear a commotion coming from the kitchen. "At this rate, we won't have any time left to put the wreaths up in the dining hall!" Resolute in your quest, you investigate.

"If only we hadn't switched to the new inventory management system right before Christmas!" another Elf exclaims. You ask what's going on.

The Elves in the kitchen explain the situation: because of their complicated new inventory management system, they can't figure out which of their ingredients are fresh and which are spoiled. When you ask how it works, they give you a copy of their database (your puzzle input).

The database operates on ingredient IDs. It consists of a list of fresh ingredient ID ranges, a blank line, and a list of available ingredient IDs. For example:
```
3-5
10-14
16-20
12-18

1
5
8
11
17
32
```
The fresh ID ranges are inclusive: the range `3-5` means that ingredient IDs `3`, `4`, and `5` are all fresh. The ranges can also overlap; an ingredient ID is fresh if it is in any range.

The Elves are trying to determine which of the available ingredient IDs are fresh. In this example, this is done as follows:

Ingredient ID `1` is spoiled because it does not fall into any range.
Ingredient ID `5` is fresh because it falls into range `3-5`.
Ingredient ID `8` is spoiled.
Ingredient ID `11` is fresh because it falls into range `10-14`.
Ingredient ID `17` is fresh because it falls into range `16-20` as well as range `12-18`.
Ingredient ID `32` is spoiled.
So, in this example, `3` of the available ingredient IDs are fresh.

Process the database file from the new inventory management system. How many of the available ingredient IDs are fresh?

In [10]:
class DayFivePartOne(override val testResult: Int = 3) : Problem<Int>("day5.txt") {
    override fun solve(lines: Sequence<String>): Int {
        val lineList = lines.toList()
        val okayIds = lineList
            .takeWhile { it.isNotBlank() }
            .map { it.substringBefore("-").toLong() to it.substringAfter("-").toLong() }
        val ingredients = lineList.dropWhile { it.contains("-") || it.isBlank() }.map { it.toLong() }
        val freshIds = mutableSetOf<Long>()
        okayIds.forEach { (start, end) ->
            ingredients.forEach { ingredient ->
                if (ingredient in start..end) {
                    freshIds.add(ingredient)
                }
            }
        }
        return freshIds.size
    }
}

DayFivePartOne().run()

Test results: Expected=3 Actual=3
Solution: 613


### Part 2
The Elves start bringing their spoiled inventory to the trash chute at the back of the kitchen.

So that they can stop bugging you when they get new inventory, the Elves would like to know all of the IDs that the fresh ingredient ID ranges consider to be fresh. An ingredient ID is still considered fresh if it is in any range.

Now, the second section of the database (the available ingredient IDs) is irrelevant. Here are the fresh ingredient ID ranges from the above example:
```
3-5
10-14
16-20
12-18
```
The ingredient IDs that these ranges consider to be fresh are `3`, `4`, `5`, `10`, `11`, `12`, `13`, `14`, `15`, `16`, `17`, `18`, `19`, and `20`. So, in this example, the fresh ingredient ID ranges consider a total of `14` ingredient IDs to be fresh.

Process the database file again. How many ingredient IDs are considered to be fresh according to the fresh ingredient ID ranges?

In [11]:
class DayFivePartTwo(override val testResult: Long = 14) : Problem<Long>("day5.txt") {
    override fun solve(lines: Sequence<String>): Long {
        val okayIds = lines
            .takeWhile { it.isNotBlank() }
            .map { it.substringBefore("-").toLong() to it.substringAfter("-").toLong() }
            .sortedBy { it.first }
        val condensedFreshIds: MutableSet<Pair<Long, Long>> = mutableSetOf()
        okayIds.forEach { freshRange ->
            val toUpdate = condensedFreshIds.firstOrNull { condensedRange -> freshRange.overlaps(condensedRange) }
            if (toUpdate == null) {
                condensedFreshIds.add(freshRange)
            } else {
                condensedFreshIds.remove(toUpdate)
                condensedFreshIds.add(min(freshRange.first, toUpdate.first) to max(freshRange.second, toUpdate.second))
            }
        }
        return condensedFreshIds.sumOf { it.second - it.first + 1 }
    }

    private fun Pair<Long, Long>.overlaps(other: Pair<Long, Long>): Boolean =
        this.first in other.first..other.second || this.second in other.first..other.second
}

DayFivePartTwo().run()

Test results: Expected=14 Actual=14
Solution: 336495597913098


## Day 6: Cephalopod math
### Part 1
After helping the Elves in the kitchen, you were taking a break and helping them re-enact a movie scene when you over-enthusiastically jumped into the garbage chute!

A brief fall later, you find yourself in a garbage smasher. Unfortunately, the door's been magnetically sealed.

As you try to find a way out, you are approached by a family of cephalopods! They're pretty sure they can get the door open, but it will take some time. While you wait, they're curious if you can help the youngest cephalopod with her math homework.

Cephalopod math doesn't look that different from normal math. The math worksheet (your puzzle input) consists of a list of problems; each problem has a group of numbers that need to be either added (`+`) or multiplied (`*`) together.

However, the problems are arranged a little strangely; they seem to be presented next to each other in a very long horizontal list. For example:
```
123 328  51 64
 45 64  387 23
  6 98  215 314
*   +   *   +
```
Each problem's numbers are arranged vertically; at the bottom of the problem is the symbol for the operation that needs to be performed. Problems are separated by a full column of only spaces. The left/right alignment of numbers within each problem can be ignored.

So, this worksheet contains four problems:
```
123 * 45 * 6 = 33210
328 + 64 + 98 = 490
51 * 387 * 215 = 4243455
64 + 23 + 314 = 401
```
To check their work, cephalopod students are given the grand total of adding together all of the answers to the individual problems. In this worksheet, the grand total is `33210 + 490 + 4243455 + 401 = 4277556`.

Of course, the actual worksheet is much wider. You'll need to make sure to unroll it completely so that you can read the problems clearly.

Solve the problems on the math worksheet. What is the grand total found by adding together all of the answers to the individual problems?

In [12]:
class DaySixPartOne(override val testResult: Long = 4277556) : Problem<Long>("day6.txt") {
    override fun solve(lines: Sequence<String>): Long {
        val data = lines.map { it.trim().split("\\s+".toRegex()) }.toList()
        return data.rotate().sumOf { row ->
            return@sumOf when (row.first()) {
                "+" -> row.subList(1, row.size).sumOf { it.toLong() }
                "*" -> {
                    row.subList(1, row.size).map { it.toLong() }.product()
                }
                else -> 0L
            }
        }
    }

    private fun <T> List<List<T>>.rotate(): List<List<T>> {
        val numCols = this[0].size
        val numRows = this.size

        val rotated = List(numCols) { mutableListOf<T>() }

        for (i in 0 until numRows) {
            for (j in 0 until numCols) {
                rotated[j].add(0, this[i][j])
            }
        }

        return rotated.map { it.toList() }
    }

    private fun List<Long>.product(): Long {
        var out = this.first()
        this.subList(1, this.size).forEach { out *= it }
        return out
    }
}

DaySixPartOne().run()

Test results: Expected=4277556 Actual=4277556
Solution: 4309240495780


### Part 2
The big cephalopods come back to check on how things are going. When they see that your grand total doesn't match the one expected by the worksheet, they realize they forgot to explain how to read cephalopod math.

Cephalopod math is written right-to-left in columns. Each number is given in its own column, with the most significant digit at the top and the least significant digit at the bottom. (Problems are still separated with a column consisting only of spaces, and the symbol at the bottom of the problem is still the operator to use.)

Here's the example worksheet again:
```
123 328  51 64
 45 64  387 23
  6 98  215 314
*   +   *   +
```
Reading the problems right-to-left one column at a time, the problems are now quite different:

The rightmost problem is `4 + 431 + 623 = 1058`
The second problem from the right is `175 * 581 * 32 = 3253600`
The third problem from the right is `8 + 248 + 369 = 625`
Finally, the leftmost problem is `356 * 24 * 1 = 8544`
Now, the grand total is `1058 + 3253600 + 625 + 8544 = 3263827`.

Solve the problems on the math worksheet again. What is the grand total found by adding together all of the answers to the individual problems?

In [13]:
class DaySixPartTwo(override val testResult: Long = 3263827) : Problem<Long>("day6.txt") {
    override fun solve(lines: Sequence<String>): Long {
        val input = lines.toList()
        val longest = input.maxOf { it.length }
        val data = input.map { line ->
            var out = line.toMutableList()
            repeat(longest-line.length) {
                out.add(' ')
            }
            out.reversed().joinToString("")
        }

        val noOp = data.dropLast(1)
        val operators = data.takeLast(1)

        var sum = 0L
        val numbers = mutableListOf<Long>()
        for (col in 0..data.first().length - 1) {
            val num = noOp.map { it[col] }.joinToString("").trim()
            if (num.isNotBlank()) numbers.add(num.toLong())
            when (operators[0].getOrNull(col)) {
                '+' -> {
                    sum += numbers.sum()
                    numbers.clear()
                }
                '*' -> {
                    sum += numbers.product()
                    numbers.clear()
                }
            }
        }
        return sum
    }

    private fun List<Long>.product(): Long {
        var product = 1L
        this.forEach { product *= it }
        return product
    }
}

DaySixPartTwo().run()

Test results: Expected=3263827 Actual=3263827
Solution: 9170286552289


## Day 7: Tachyon Manifold
### Part 1
You thank the cephalopods for the help and exit the trash compactor, finding yourself in the familiar halls of a North Pole research wing.

Based on the large sign that says "teleporter hub", they seem to be researching teleportation; you can't help but try it for yourself and step onto the large yellow teleporter pad.

Suddenly, you find yourself in an unfamiliar room! The room has no doors; the only way out is the teleporter. Unfortunately, the teleporter seems to be leaking magic smoke.

Since this is a teleporter lab, there are lots of spare parts, manuals, and diagnostic equipment lying around. After connecting one of the diagnostic tools, it helpfully displays error code 0H-N0, which apparently means that there's an issue with one of the tachyon manifolds.

You quickly locate a diagram of the tachyon manifold (your puzzle input). A tachyon beam enters the manifold at the location marked S; tachyon beams always move downward. Tachyon beams pass freely through empty space (.). However, if a tachyon beam encounters a splitter (^), the beam is stopped; instead, a new tachyon beam continues from the immediate left and from the immediate right of the splitter.

For example:
```
.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
```
In this example, the incoming tachyon beam (|) extends downward from S until it reaches the first splitter:
```
.......S.......
.......|.......
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
```
At that point, the original beam stops, and two new beams are emitted from the splitter:
```
.......S.......
.......|.......
......|^|......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
```
Those beams continue downward until they reach more splitters:
```
.......S.......
.......|.......
......|^|......
......|.|......
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
```
At this point, the two splitters create a total of only three tachyon beams, since they are both dumping tachyons into the same place between them:
```
.......S.......
.......|.......
......|^|......
......|.|......
.....|^|^|.....
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
```
This process continues until all of the tachyon beams reach a splitter or exit the manifold:
```
.......S.......
.......|.......
......|^|......
......|.|......
.....|^|^|.....
.....|.|.|.....
....|^|^|^|....
....|.|.|.|....
...|^|^|||^|...
...|.|.|||.|...
..|^|^|||^|^|..
..|.|.|||.|.|..
.|^|||^||.||^|.
.|.|||.||.||.|.
|^|^|^|^|^|||^|
|.|.|.|.|.|||.|
```
To repair the teleporter, you first need to understand the beam-splitting properties of the tachyon manifold. In this example, a tachyon beam is split a total of `21` times.

Analyze your manifold diagram. How many times will the beam be split?

In [14]:
class DaySevenPartOne(override val testResult: Int = 21) : Problem<Int>("day7.txt") {
    override fun solve(lines: Sequence<String>): Int {
        val data = lines.toList()
        var beamPositions = setOf<Int>()
        var numSplits = 0
        for (row in 0 until data.size) {
            val rowBeamPositions = beamPositions.toMutableSet()
            data[row].forEachIndexed { col, char ->
                if (char == 'S') {
                    rowBeamPositions.add(col)
                }

                if (char == '^' && beamPositions.contains(col)) {
                    rowBeamPositions.remove(col)
                    rowBeamPositions.addAll(listOf(col - 1, col + 1))
                    numSplits++
                }
            }
            beamPositions = rowBeamPositions
        }
        return numSplits
    }
}

DaySevenPartOne().run()

Test results: Expected=21 Actual=21
Solution: 1660


### Part 2
With your analysis of the manifold complete, you begin fixing the teleporter. However, as you open the side of the teleporter to replace the broken manifold, you are surprised to discover that it isn't a classical tachyon manifold - it's a quantum tachyon manifold.

With a quantum tachyon manifold, only a single tachyon particle is sent through the manifold. A tachyon particle takes both the left and right path of each splitter encountered.

Since this is impossible, the manual recommends the many-worlds interpretation of quantum tachyon splitting: each time a particle reaches a splitter, it's actually time itself which splits. In one timeline, the particle went left, and in the other timeline, the particle went right.

To fix the manifold, what you really need to know is the number of timelines active after a single particle completes all of its possible journeys through the manifold.

In the above example, there are many timelines. For instance, there's the timeline where the particle always went left:
```
.......S.......
.......|.......
......|^.......
......|........
.....|^.^......
.....|.........
....|^.^.^.....
....|..........
...|^.^...^....
...|...........
..|^.^...^.^...
..|............
.|^...^.....^..
.|.............
|^.^.^.^.^...^.
|..............
```
Or, there's the timeline where the particle alternated going left and right at each splitter:
```
.......S.......
.......|.......
......|^.......
......|........
......^|^......
.......|.......
.....^|^.^.....
......|........
....^.^|..^....
.......|.......
...^.^.|.^.^...
.......|.......
..^...^|....^..
.......|.......
.^.^.^|^.^...^.
......|........
```
Or, there's the timeline where the particle ends up at the same point as the alternating timeline, but takes a totally different path to get there:
```
.......S.......
.......|.......
......|^.......
......|........
.....|^.^......
.....|.........
....|^.^.^.....
....|..........
....^|^...^....
.....|.........
...^.^|..^.^...
......|........
..^..|^.....^..
.....|.........
.^.^.^|^.^...^.
......|........
```
In this example, in total, the particle ends up on 40 different timelines.

Apply the many-worlds interpretation of quantum tachyon splitting to your manifold diagram. In total, how many different timelines would a single tachyon particle end up on?

In [15]:
class DaySevenPartTwo(override val testResult: Long = 40) : Problem<Long>("day7.txt") {
    override fun solve(lines: Sequence<String>): Long {
        val data = lines.toList().filterNot { line -> line.all { it == '.' } }
        val start = data.first().indexOfFirst { it == 'S' }
        var paths: Map<Int, Long> = mapOf(start to 1L)
        data.drop(1).forEach { line ->
            val lastPaths = paths.toMap()
            val currentPaths = paths.toMutableMap()
            line.forEachIndexed { index, char ->
                lastPaths[index]?.let {
                    if (char == '^' && it > 0) {
                        currentPaths[index] = 0
                        currentPaths[index - 1] = (lastPaths[index] ?: 0) + (currentPaths[index - 1] ?: 0)
                        currentPaths[index + 1] = (lastPaths[index] ?: 0) + (currentPaths[index + 1] ?: 0)
                    }
                }
            }
            paths = currentPaths
        }

        return paths.entries.sumOf { it.value }
    }
}

DaySevenPartTwo().run()

Test results: Expected=40 Actual=40
Solution: 305999729392659


## Day 8
### Part 1
Equipped with a new understanding of teleporter maintenance, you confidently step onto the repaired teleporter pad.

You rematerialize on an unfamiliar teleporter pad and find yourself in a vast underground space which contains a giant playground!

Across the playground, a group of Elves are working on setting up an ambitious Christmas decoration project. Through careful rigging, they have suspended a large number of small electrical junction boxes.

Their plan is to connect the junction boxes with long strings of lights. Most of the junction boxes don't provide electricity; however, when two junction boxes are connected by a string of lights, electricity can pass between those two junction boxes.

The Elves are trying to figure out which junction boxes to connect so that electricity can reach every junction box. They even have a list of all of the junction boxes' positions in 3D space (your puzzle input).

For example:
```
162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689
```
This list describes the position of 20 junction boxes, one per line. Each position is given as `X,Y,Z` coordinates. So, the first junction box in the list is at `X=162`, `Y=817`, `Z=812`.

To save on string lights, the Elves would like to focus on connecting pairs of junction boxes that are as close together as possible according to straight-line distance. In this example, the two junction boxes which are closest together are `162,817,812` and `425,690,689`.

By connecting these two junction boxes together, because electricity can flow between them, they become part of the same circuit. After connecting them, there is a single circuit which contains two junction boxes, and the remaining 18 junction boxes remain in their own individual circuits.

Now, the two junction boxes which are closest together but aren't already directly connected are `162,817,812` and `431,825,988`. After connecting them, since `162,817,812` is already connected to another junction box, there is now a single circuit which contains three junction boxes and an additional 17 circuits which contain one junction box each.

The next two junction boxes to connect are `906,360,560` and `805,96,715` . After connecting them, there is a circuit containing 3 junction boxes, a circuit containing 2 junction boxes, and 15 circuits which contain one junction box each.

The next two junction boxes are `431,825,988` and `425,690,689`. Because these two junction boxes were already in the same circuit, nothing happens!

This process continues for a while, and the Elves are concerned that they don't have enough extension cables for all these circuits. They would like to know how big the circuits will be.

After making the ten shortest connections, there are 11 circuits: one circuit which contains 5 junction boxes, one circuit which contains 4 junction boxes, two circuits which contain 2 junction boxes each, and seven circuits which each contain a single junction box. Multiplying together the sizes of the three largest circuits (5, 4, and one of the circuits of size 2) produces 40.

Your list contains many junction boxes; connect together the 1000 pairs of junction boxes which are closest together. Afterward, what do you get if you multiply together the sizes of the three largest circuits?

In [74]:
class DayEightPartOne(override val testResult: Int = 40) : Problem<Int>("day8.txt") {
    override fun solve(lines: Sequence<String>): Int {
        val coords = lines.toList().map { line ->
            val items = line.split(",")
            Coord(x = items[0].toLong(), y = items[1].toLong(), z = items[2].toLong())
        }
        val mappings = coords.mapIndexed { index, coord ->
            Mapping(coord).also {
                it.addDistances(coords.subList(index + 1, coords.size))
            }
        }
        val groups: MutableList<MutableSet<Coord>> = mutableListOf()
        repeat(if (isTesting) 10 else 1000) {
            val min = mappings.minBy { it.minDistance()?.value ?: Long.MAX_VALUE }
            val other = min.minDistance()!!
            val matchingGroups = groups.filter { it.contains(min.coord) || it.contains(other.key) }
            when (matchingGroups.size) {
                0 -> {
                    groups.add(mutableSetOf(min.coord, other.key))
                }
                1 -> {
                    groups[groups.indexOf(matchingGroups[0])].let {
                        it.add(min.coord)
                        it.add(other.key)
                    }
                }
                else -> {
                    val newGroup = matchingGroups.flatMap { it }.toMutableSet()
                    newGroup.add(min.coord)
                    newGroup.add(other.key)
                    groups.removeAll(matchingGroups)
                    groups.add(newGroup)
                }
            }
            min.remove(other.key)
        }

        var out = 1
        groups.sortedByDescending { it.size }.take(3).forEach { out *= it.size }
        return out
    }

    data class Coord(val x: Long, val y: Long, val z: Long) {
        fun distanceTo(other: Coord): Long = (this.x - other.x).squared() + (this.y - other.y).squared() + (this.z - other.z).squared()
        private fun Long.squared() = this * this
        override fun toString(): String = "($x, $y, $z)"
    }

    data class Mapping(val coord: Coord) {
        private val mappings: MutableMap<Coord, Long> = mutableMapOf()
        fun addDistances(coords: List<Coord>) {
            coords.forEach { other ->
                if (coord != other) {
                    mappings[other] = coord.distanceTo(other)
                }
            }
        }

        fun remove(coord: Coord) {
            mappings.remove(coord)
        }

        fun minDistance(): Map.Entry<Coord, Long>? = if (mappings.isNotEmpty()) mappings.minBy { it.value } else null
    }
}

DayEightPartOne().run()

Test results: Expected=40 Actual=40
Solution: 135169


### Part 2
The Elves were right; they definitely don't have enough extension cables. You'll need to keep connecting junction boxes together until they're all in one large circuit.

Continuing the above example, the first connection which causes all of the junction boxes to form a single circuit is between the junction boxes at 216,146,977 and 117,168,530. The Elves need to know how far those junction boxes are from the wall so they can pick the right extension cable; multiplying the X coordinates of those two junction boxes (216 and 117) produces 25272.

Continue connecting the closest unconnected pairs of junction boxes together until they're all in the same circuit. What do you get if you multiply together the X coordinates of the last two junction boxes you need to connect?

In [89]:
class DayEightPartTwo(override val testResult: Long = 25272) : Problem<Long>("day8.txt") {
    override fun solve(lines: Sequence<String>): Long {
        val coords = lines.toList().map { line ->
            val items = line.split(",")
            Coord(x = items[0].toLong(), y = items[1].toLong(), z = items[2].toLong())
        }
        val mappings = coords.mapIndexed { index, coord ->
            Mapping(coord).also {
                it.addDistances(coords.subList(index + 1, coords.size))
            }
        }
        val groups: MutableList<MutableSet<Coord>> = mutableListOf()
        var lastPair: Pair<Coord, Coord> = Coord(0L, 0L, 0L) to Coord(0L, 0L, 0L)
        do {
            val hasConnections = mappings.filter { it.hasConnectionsAvailable() }
            val min = hasConnections.minBy { it.minDistance()?.value ?: Long.MAX_VALUE }
            val other = min.minDistance()!!
            val matchingGroups = groups.filter { it.contains(min.coord) || it.contains(other.key) }
            when (matchingGroups.size) {
                0 -> {
                    groups.add(mutableSetOf(min.coord, other.key))
                }
                1 -> {
                    groups[groups.indexOf(matchingGroups[0])].let {
                        it.add(min.coord)
                        it.add(other.key)
                    }
                }
                else -> {
                    val newGroup = (matchingGroups[0] + matchingGroups[1]).toMutableSet()
                    newGroup.add(min.coord)
                    newGroup.add(other.key)
                    groups.removeAll(matchingGroups)
                    groups.add(newGroup)
                }
            }
            min.remove(other.key)

            lastPair = min.coord to other.key
        } while (groups[0].size != mappings.size)

        return lastPair.first.x * lastPair.second.x
    }

    data class Coord(val x: Long, val y: Long, val z: Long) {
        fun distanceTo(other: Coord): Long = (this.x - other.x).squared() + (this.y - other.y).squared() + (this.z - other.z).squared()
        private fun Long.squared() = this * this
        override fun toString(): String = "($x, $y, $z)"
    }

    data class Mapping(val coord: Coord) {
        private val mappings: MutableMap<Coord, Long> = mutableMapOf()
        fun addDistances(coords: List<Coord>) {
            coords.forEach { other ->
                if (coord != other) {
                    mappings[other] = coord.distanceTo(other)
                }
            }
        }

        fun remove(coord: Coord) {
            mappings.remove(coord)
        }

        fun minDistance(): Map.Entry<Coord, Long>? = if (mappings.isNotEmpty()) mappings.minBy { it.value } else null

        fun hasConnectionsAvailable(): Boolean = mappings.isNotEmpty()
    }
}

DayEightPartTwo().run()

Test results: Expected=25272 Actual=25272
Solution: 302133440
